# Example 4 Functional in R

### This example would use the 2-stage model in BI IPF project to illustrate how to do functional in R
#### 1. Loading in the relevant packages and reading in data

In [1]:
library(readr)
library(dplyr)
library(ROCR)
library(PRROC)
library(data.table)

inpath <- 'D:/Project_2016/Spark/template_code/template_Data/'
outpath <- 'D:/Project_2017/Training_0331/Functional_results/'
mdata_fnm <- 'Model_Tempdata_2stage_FoldID.csv'
rdata_fnm <- 'Rep_Tempdata_2stage_FoldID.csv'

mdata <- read_csv(paste0(inpath, mdata_fnm))
rdata <- read_csv(paste0(inpath, rdata_fnm))


Attaching package: 'dplyr'

The following objects are masked from 'package:stats':

    filter, lag

The following objects are masked from 'package:base':

    intersect, setdiff, setequal, union

Loading required package: gplots

Attaching package: 'gplots'

The following object is masked from 'package:stats':

    lowess


Attaching package: 'data.table'

The following objects are masked from 'package:dplyr':

    between, first, last



In [2]:
dim(mdata)
dim(rdata)

[1] 40200    16

[1] 130600     16

#### We will do 5-fold outer cross-evaluation and 4-fold inner cross-evaluation

In [7]:
ofold_seq <- c(0, 1, 2, 3, 4)
ifold_seq <- c(0, 1, 2, 3)

#### 2. Defining 3 functions to help us to access the results
#### Function 1: to summarize the results from inner cross-evaluation

In [3]:
pred_dt <- function(pred_all){
  function(dt){
    pred_ls <- lapply(pred_all, function(x)x[[dt]])
    return(rbindlist(pred_ls))
  }
}

#### Function 2: to subset the data for 2nd stage model

In [4]:
subset_dt <- function(dt, threshold){
  function(all_dt){
    dt$patid <- as.character(dt$patid)
    all_dt$patid <- as.character(all_dt$patid)
    
    sublog <- dt %>% 
      filter(prob >= threshold) %>% 
      mutate(log_prob = log10(prob)) %>%
      inner_join(all_dt, by="patid") %>% 
      select(-c(label.x, prob, patid)) %>% 
      rename(label = label.y)
    
    return(sublog)
  }
}

#### Function 3: to summarize the results from cross-evaluation and export

In [5]:
predict_all <- function(pred_all, name){
  aucobj <- prediction(pred_all$prob, pred_all$label)
  auc_all <- performance(aucobj, 'auc')@y.values
  aupr_all <- pr.curve(scores.class0 = pred_all$prob, weights.class0 = pred_all$label)$auc.integral
  perf_all <- data.frame(AUC=auc_all[[1]], AUPR=aupr_all)
  write_csv(perf_all, paste0(outpath, name, '_auc_aupr.csv'))
}


#### 3. Do the nested cross-evaluation with standard LR

In [8]:
tm1 <- Sys.time()
# Outer loops
pred2nd_all <- lapply(ofold_seq, function(x){
  
  # Split the data into training, test and rep for each outer loop   
  tr <- mdata %>% filter(OuterFoldID !=x) %>% select(-c(matched_patient_id))
  ts <- mdata %>% filter(OuterFoldID ==x) %>% select(-c(OuterFoldID, InnerFoldID,matched_patient_id, patid))
  rep <- rdata %>% filter(OuterFoldID ==x) %>% select(-c(OuterFoldID, InnerFoldID, matched_patient_id, patid))
  
  # These dataset only contains InnerFoldID, OuterFoldID, Patient_ID and Matched_ID
  tr_for2nd <- mdata %>% filter(OuterFoldID !=x) %>% select(-c(OuterFoldID, InnerFoldID,matched_patient_id))
  ts_for2nd <- mdata %>% filter(OuterFoldID ==x) %>% select(-c(OuterFoldID, InnerFoldID,matched_patient_id))
  rep_for2nd <- rdata %>% filter(OuterFoldID ==x) %>% select(-c(OuterFoldID, InnerFoldID,matched_patient_id))
  
  # Inner loops  
  pred1st_all <- lapply(ifold_seq, function(y){
    
    # Split data into cv_training, cv_test  
    cv_tr <- tr %>% filter(InnerFoldID !=y) %>% select(-c(OuterFoldID, InnerFoldID, patid))
    cv_ts <- tr %>% filter(InnerFoldID ==y) %>% select(-c(OuterFoldID, InnerFoldID, patid))
    cv_ts_patid <- tr %>% filter(InnerFoldID ==y) %>% select(patid)
    
    # Train the standard LR on cv_training data  
    cv_lr <- glm(label~., data = cv_tr, family=binomial)
      
    # Predict on all the datasets
    pred_cvtr <- predict(cv_lr, cv_tr, type='response')
    pred_cvts <- predict(cv_lr, cv_ts, type="response" )
    pred_ts <- predict(cv_lr, ts, type="response" )
    pred_rep <- predict(cv_lr, rep, type="response" )
    
    # Generate the dataframe contains predicted scores and labels 
    pred_lb_cvtr <- data.frame(label=cv_tr$label, prob=pred_cvtr)
    pred_lb_cvts <- data.frame(label=cv_ts$label, prob=pred_cvts)
      
    # Export the performance for each inner cross-evaluation  
    predict_all(pred_lb_cvtr, paste0('cvtr_outer', x, '_inner', y))
    predict_all(pred_lb_cvts, paste0('cvts_outer', x, '_inner', y))
    
    # Summarize the results into a list  
    pred_ls <- list(
      cvts = data.frame(label=cv_ts$label, patid=cv_ts_patid, prob=pred_cvts),
      ts = data.frame(label=ts$label, patid=ts_for2nd$patid, prob=pred_ts),
      rep = data.frame(label=rep$label, patid=rep_for2nd$patid, prob=pred_rep)
    )
    
    # The function would return the list
    return(pred_ls)
    
  })
  
  # Call function 1 to summarize the predictions accross inner loops  
  pred_tr <- pred_dt(pred1st_all)('cvts')
  pred_ts <- pred_dt(pred1st_all)('ts')
  pred_rep <- pred_dt(pred1st_all)('rep')
  
  # Averaging the predictions for test and rep data  
  avg_pred_ts <- aggregate(pred_ts, list(pred_ts$patid), mean)[, -1]
  avg_pred_rep <- aggregate(pred_rep, list(pred_rep$patid), mean)[, -1]
  
  # Find the top 25% score based on the combined predictions on training data
  pct <- pred_tr %>% 
    filter(label==1) %>% 
    summarise(thres=quantile(prob, probs=0.75))
  threshold <- pct$thres
  
  # Subset the training, test and rep data by function 2
  pred_tr <- data.frame(pred_tr)
  subtr <- subset_dt(pred_tr, threshold)(tr_for2nd)
  subts <- subset_dt(avg_pred_ts, threshold)(ts_for2nd)
  subrep <- subset_dt(avg_pred_rep, threshold)(rep_for2nd)

  # Train teh 2nd stage model and predict
  lr2nd <- glm(label~., data = subtr, family=binomial)
  pred_subtr <- predict(lr2nd, subtr, type="response" )
  pred_subts <- predict(lr2nd, subts, type="response" )
  pred_subrep <- predict(lr2nd, subrep, type="response" )
  
  # Summarize the results into a list  
  pred_subls <- list(
    subtr = data.frame(label=subtr$label, prob=pred_subtr),
    subts = data.frame(label=subts$label, prob=pred_subts),
    subrep = data.frame(label=subrep$label, prob=pred_subrep)
  )
  
  # Return the list  
  return(pred_subls)
  
})
print( Sys.time()-tm1)

Time difference of 34.9805 secs


#### 4. Summarize the results from 2nd stage model by recalling function 1

In [9]:
pred_subtr <- pred_dt(pred2nd_all)('subtr')
pred_subts <- pred_dt(pred2nd_all)('subts')
pred_subrep <- pred_dt(pred2nd_all)('subrep')

#### 5. Export the summarized results by recalling function 3

In [10]:
predict_all(pred_subtr, 'subtr')
predict_all(pred_subts, 'subts')
pred_tsrep <- rbind(pred_subts, pred_subrep)
predict_all(pred_tsrep, 'subtsrep')

#### 6. Summarize the cv performance by `do.call`

In [11]:
# Get all the files in a directory
files_cvtr <- list.files(path = outpath, pattern = 'cvtr_*', full.names = T, recursive = F)
files_cvts <- list.files(path = outpath, pattern = 'cvts_*', full.names = T, recursive = F)

# rbind the performance from cv_training
perf_cvtr <- do.call('rbind', lapply(files_cvtr, function(x){
  read_csv(x)
}))

# rbind the performance from cv_test
perf_cvts <- do.call('rbind', lapply(files_cvts, function(x){
  read_csv(x)
}))

# cbind them together 
perf_r <- data.frame(perf_cvtr, perf_cvts)

# Calculate the mean by column
perf_meanr <- colMeans(perf_r)

# Export it 
perf_r_f <- rbind(perf_r, perf_meanr)
names(perf_r_f) <- c('cvtr_auc', 'cvtr_aupr', 'cvts_auc', 'cvts_aupr')
name <- c(seq(1:20), "Mean")
perf_r_f2 <- cbind(name, perf_r_f)
write_csv(perf_r_f2, paste0(outpath, 'cv_R_auc_aupr.csv'))